In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 k

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-05-25 00:17:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2022-05-25 00:17:12 (6.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ='https://globalcropyieldanalysis.s3.amazonaws.com/yield.csv'
spark.sparkContext.addFile(url)
yield_df = spark.read.csv(SparkFiles.get("yield.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ='https://globalcropyieldanalysis.s3.amazonaws.com/temp.csv'
spark.sparkContext.addFile(url)
temp_df = spark.read.csv(SparkFiles.get("temp.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ='https://globalcropyieldanalysis.s3.amazonaws.com/rainfall.csv'
spark.sparkContext.addFile(url)
rain_df = spark.read.csv(SparkFiles.get("rainfall.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ='https://globalcropyieldanalysis.s3.amazonaws.com/pesticides.csv'
spark.sparkContext.addFile(url)
pesticides_df = spark.read.csv(SparkFiles.get("pesticides.csv"), sep=",", header=True, inferSchema=True)

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ='https://globalcropyieldanalysis.s3.amazonaws.com/Crop_recommendation.csv'
spark.sparkContext.addFile(url)
crop_df = spark.read.csv(SparkFiles.get("Crop_recommendation.csv"), sep=",", header=True, inferSchema=True)

In [14]:
# Store environmental variable
from getpass import getpass
password = getpass('password')
# Configure settings for RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://project.cqupc8fzrokq.us-east-1.rds.amazonaws.com:5432/Global_Crop_Yield_Analysis"
config = {"user":"sohrabrezaei",
          "password": password,
          "driver":"org.postgresql.Driver"}

password··········


In [15]:
# Write DataFrame to yield table in RDS
yield_df.write.jdbc(url=jdbc_url, table='Yield', mode=mode, properties=config)

In [16]:
# Write DataFrame to rain table in RDS
rain_df.write.jdbc(url=jdbc_url, table='Rainfall', mode=mode, properties=config)

In [17]:
# Write DataFrame to pesticides table in RDS
pesticides_df.write.jdbc(url=jdbc_url, table='Pesticides', mode=mode, properties=config)

In [18]:
# Write DataFrame to temp table in RDS
temp_df.write.jdbc(url=jdbc_url, table='Temp', mode=mode, properties=config)

In [25]:
# Write DataFrame to crop table in RDS
crop_df.write.jdbc(url=jdbc_url, table='Crop', mode=mode, properties=config)